In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import sklearn as sk
import random
import math as mat
#df.info()

In [3]:
#блок функций
def rmse(y, y_pred):
    return mat.sqrt(((y-y_pred)**2 ).sum()/len(y_pred)) 
def get_r2(x, y):
    slope, intercept = np.polyfit(x, y, 1)
    r_squared = 1 - (sum((y - (slope * x + intercept))**2) / ((len(y) - 1) * np.var(y, ddof=1)))
    return r_squared
#проверка типов фич
def is_floating(x):
    return (x - x.astype(int)).abs().sum()>0

def is_boolean(x):
    return x.unique().shape[0] == 2

def is_const(x):
    return x.unique().shape[0] == 1

#pd.DataFrame({
    #'float': df.apply(is_floating, axis=0),
    #'boolean': df.apply(is_boolean, axis=0),
    #'const': df.apply(is_const, axis=0)
    
#})
#проверка корреляции (по Пирсону) фич
def is_cor(X):
    iscor = X.corr()['Target'][X.corr()['Target'] < 1].abs()
    iscor = iscor.sort_values(ascending=False)
    return iscor

In [4]:
#чтение данных
path = r'C:\Users\SUB\Desktop\test folder\dataset.csv'#
df = pd.read_csv(path)

#удаление лишних столбцов
df = df.drop('Post Promotion Status', 1)
df = df.drop('Unnamed: 0', 1)
df = df.sample(frac=1)  #перемещиваем

#нормировка кроме таргета и булевых
for c in df.columns: 
    if((c!='Target') & (is_boolean(df[c])== False)):
        mean, std = df[c].mean(), df[c].std() 
        df[c]=(df[c] - mean)/std 
df.to_csv(r'C:\Users\SUB\Desktop\test folder\datasetOUT.csv')
#df[df.columns[52]]
#df[df.columns[52]].to_excel(r'C:\Users\SUB\Desktop\test folder\dataout.xlsx', startrow=3, index=False)# выгрузка в эксель

In [26]:
#проверка удаления таргетов
#for i in range(1,100):
    #print(i, len(df[df.Target > i])/40949)
#df = df.drop(df[df.Target > 100].index) #удаление Таргет>100
print(df.columns[0:52].values)
#.to_excel(r'C:\Users\SUB\Desktop\test folder\dataout.xlsx', startrow=3, index=False)# выгрузка в эксель

['Page Popularity' 'Page Checkins' 'Page talking about' 'Page Category'
 'extra_0' 'extra_1' 'extra_2' 'extra_3' 'extra_4' 'extra_5' 'extra_6'
 'extra_7' 'extra_8' 'extra_9' 'extra_10' 'extra_11' 'extra_12' 'extra_13'
 'extra_14' 'extra_15' 'extra_16' 'extra_17' 'extra_18' 'extra_19'
 'extra_20' 'extra_21' 'extra_22' 'extra_23' 'extra_24' 'CC1' 'CC2' 'CC3'
 'CC4' 'CC5' 'Base Time' 'Post Length' 'Post Share Count' 'H Local'
 'published_weekday_0' 'published_weekday_1' 'published_weekday_2'
 'published_weekday_3' 'published_weekday_4' 'published_weekday_5'
 'published_weekday_6' 'base_weekday_0' 'base_weekday_1' 'base_weekday_2'
 'base_weekday_3' 'base_weekday_4' 'base_weekday_5' 'base_weekday_6']


In [4]:
df=pd.read_csv(r'C:\Users\SUB\Desktop\test folder\datasetOUT.csv')
df = df.drop('Unnamed: 0', 1)
#df = df.drop(df[df.Target > 100].index)
fold = np.array_split(df, 5)
iscor=is_cor(df)#корреляция столбцов
#print(iscor)

In [61]:
#библиотека sklearn

RMSE_sum=0
R2_sum=0
for i in range(0,5):
    pdcont=df.drop(fold[4-i].index)
    features = df.columns[0:52]
    training_input = pdcont[features].values
    training_output = pdcont['Target'].values
    test_input = fold[4-i][features].values
    test_output = fold[4-i]['Target'].values
    
    linreg = LinearRegression().fit(training_input,training_output)
    #print("REGscore:",linreg.score(test_input,test_output))
    #print(linreg.coef_)
    
    print("TEST FOLD      ", i)
    print("RMSE:",rmse(test_output,linreg.predict(test_input)))           
    print("R2score:",get_r2(test_output,linreg.predict(test_input)))
    
    
    
    print("TRAIN FOLD      ", i)
    print("RMSE:",rmse(training_output,linreg.predict(training_input)))
    RMSE_sum += rmse(test_output,linreg.predict(test_input))           
    print("R2score:",get_r2(training_output,linreg.predict(training_input)))
    R2_sum +=get_r2(test_output,linreg.predict(test_input))  
print("R2 mean",R2_sum/5)
print("RMSE mean",RMSE_sum/5)

[ 1.65064326e-01 -3.06428346e-01 -2.06165540e+00 -1.78849314e-01
 -6.65827239e+00  5.44625982e+00  8.54271445e+01 -7.61371792e-01
 -1.35504182e+01 -1.42644511e+00 -6.01378704e-01 -5.83458049e+12
  4.15488867e+00  1.35600220e+00 -4.53493804e-01  4.10322746e+00
  4.91247898e+12  2.57119863e-02 -8.10815979e+00  5.40783444e+00
 -6.04001928e+00 -8.19756273e+01 -1.48366778e+00  1.61314138e+01
 -3.34640210e-01 -9.10975925e-01  2.77187483e+12 -9.55199733e-02
 -2.46280924e-01  5.16623113e+00  1.97356016e+12 -1.81885493e+12
 -7.88089238e+00 -2.41059336e+12 -4.16064453e+00 -4.39453125e-03
  2.48089600e+00  9.03808594e-01  8.24619072e+12  8.24619072e+12
  8.24619072e+12  8.24619072e+12  8.24619072e+12  8.24619072e+12
  8.24619072e+12  2.91398937e+13  2.91398937e+13  2.91398937e+13
  2.91398937e+13  2.91398937e+13  2.91398937e+13  2.91398937e+13]
TEST FOLD       0
RMSE: 22.865294655117662
R2score: 0.41244294125598524
TRAIN FOLD       0
RMSE: 30.550471264414014
R2score: 0.30983445470177784
[ 5.54416

In [6]:
def normal_equation(X, y): #вычисляет вектор весов w согласно нормальному уравнению линейной регрессии.
    a = np.dot(X.T, X) 
    b = np.dot(X.T, y) 
    res = np.linalg.solve(a, b) 
    return res
features = df.columns[0:52]

def linear_prediction(X, w): #предсказание
    return np.dot(X, w)

def gradient_step_v(X, y, w, train_ind, eta=0.01): #шаг градиента
    N = X.shape[0]                   
    x = X[train_ind]                 
    y_pred = linear_prediction(x, w) 
    rs = (y_pred - y[train_ind])     
    return  w - 2.0* eta/N*x*rs

def gradient_descent(X, y, w = 0, eta=0.1, max_iter= df.shape[0]): #градиентный спуск
    iter_num = 0
    max_iter = X.shape[0]
    while iter_num < max_iter:
        w = gradient_step_v(X, y, w, iter_num, eta)
        iter_num += 1           
    return w
                  
#stoch_grad_desc_weights = gradient_descent(df[features].values, df['Target'].values)             
#print(stoch_grad_desc_weights)
#print(rmse(df['Target'].values, linear_prediction(df[features].values, stoch_grad_desc_weights)))
#print(get_r2(df['Target'].values,linear_prediction(df[features].values, stoch_grad_desc_weights)))

In [63]:

name = ['R^2-test', 'R^2-train', 'RMSE-test', 'RMSE-train'] + df.columns[0:52].values.tolist()
fine = pd.DataFrame({
         'T1': 0,
         'T2': 0,
         'T3': 0,
         'T4': 0,
         'T5': 0,
         'E': 0,
         'STD': 0,

}, index = name)
print(fine)

                     T1  T2  T3  T4  T5  E  STD
R^2-test              0   0   0   0   0  0    0
R^2-train             0   0   0   0   0  0    0
RMSE-test             0   0   0   0   0  0    0
RMSE-train            0   0   0   0   0  0    0
Page Popularity       0   0   0   0   0  0    0
Page Checkins         0   0   0   0   0  0    0
Page talking about    0   0   0   0   0  0    0
Page Category         0   0   0   0   0  0    0
extra_0               0   0   0   0   0  0    0
extra_1               0   0   0   0   0  0    0
extra_2               0   0   0   0   0  0    0
extra_3               0   0   0   0   0  0    0
extra_4               0   0   0   0   0  0    0
extra_5               0   0   0   0   0  0    0
extra_6               0   0   0   0   0  0    0
extra_7               0   0   0   0   0  0    0
extra_8               0   0   0   0   0  0    0
extra_9               0   0   0   0   0  0    0
extra_10              0   0   0   0   0  0    0
extra_11              0   0   0   0   0 

In [68]:
rmse_test_sum = 0
r2_test_sum = 0
rmse_train_sum = 0
r2_train_sum = 0
for i in range(0,5):
    pdcont=df.drop(fold[4-i].index)
    #features = iscor.index[0:52:].values
    features = df.columns[0:52]
    training_input = pdcont[features].values
    training_output = pdcont['Target'].values
    test_input = fold[4-i][features].values
    test_output = fold[4-i]['Target'].values
    
    norm_eq_w= normal_equation(training_input, training_output)
    linreg = gradient_descent(training_input,training_output,norm_eq_w)#oбучаем
   # print("REGscore:",linreg.score(test_input,test_output))
    #print(linear_prediction(test_input, linreg))
    
    #r2 test train
    r2_test = get_r2(test_output,linear_prediction(test_input, linreg))
    r2_test_sum += r2_test
    r2_train = get_r2(training_output,linear_prediction(training_input, linreg))
    r2_train_sum += r2_train
   # 
    rmse_test = rmse(test_output,linear_prediction(test_input, linreg))
    rmse_test_sum += rmse_test
    rmse_train = rmse(training_output,linear_prediction(training_input, linreg))
    rmse_train_sum += rmse_test
    colum = [r2_test, r2_train, rmse_test, rmse_train] + linreg.tolist()
    fine[fine.columns[i]] = colum
fine.to_excel(r'C:\Users\SUB\Desktop\test folder\dataout.xlsx', index=True)
    
#     print("TEST FOLD      ", i)
#     print("RMSE:",rmse(test_output,linear_prediction(test_input, linreg)))
#     RMSE_sum += rmse(test_output,linear_prediction(test_input, linreg))           
#     print("R2score:",get_r2(test_output,linear_prediction(test_input, linreg)))
#     R2_sum +=get_r2(test_output,linear_prediction(test_input, linreg))  
    
#     print("TRAIN FOLD      ", i)
#     print("RMSE:",rmse(training_output,linear_prediction(training_input, linreg)))          
#     print("R2score:",get_r2(training_output,linear_prediction(training_input, linreg)))
    
# print("R2 mean",R2_sum/5)
# print("RMSE mean",RMSE_sum/5)

In [ ]:
import matplotlib.pyplot as plt
for c in df.columns:
    plt.scatter(df[c],df["Target"])
    plt.show()

#np.set_printoptions(threshold=np.nan) #для
